In [1]:
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
import os
import matplotlib.pyplot as plt
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [2]:
def crop_face_yawn(folder, label):
    IMG_SIZE = 145
    data = []
    
    for image in os.listdir(folder):
        image_array = cv2.imread(os.path.join(folder, image), cv2.IMREAD_COLOR)
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
        for (x, y, w, h) in faces:
            img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
            roi_color = img[y:y+h, x:x+w]
            resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
            cv2.imwrite(folder + '_mouth/' + image, resized_array)
            data.append([resized_array, label])
    
    return data;

In [2]:
def crop_face_yawn_dlib(folder, label):
    IMG_SIZE = 80
    data = []
    
    for image in os.listdir(folder):
        try:
            image_array = cv2.imread(os.path.join(folder, image), cv2.IMREAD_COLOR)
            gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 1)
            for (i, rect) in enumerate(rects):
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)

                (x, y, w, h) = face_utils.rect_to_bb(rect)
                roi = image_array[y:y+h, x:x+w]
                resized_array = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))
                cv2.imwrite(folder + '_face/' + image, resized_array)
                data.append([resized_array, label])
        except:
            continue
    return data

In [3]:
yawn_folders = ['no_yawn', 'yawn']

no_yawn = crop_face_yawn_dlib('kaggle_dataset/' + yawn_folders[0], 0)
yawn = crop_face_yawn_dlib('kaggle_dataset/' + yawn_folders[1], 1)

In [49]:
len(yawn)

510

In [3]:
yawn_folders = ['no_yawn', 'yawn']

no_yawn = crop_face_yawn('kaggle_dataset/' + yawn_folders[0], 0)
yawn = crop_face_yawn('kaggle_dataset/' + yawn_folders[1], 1)

In [11]:
combined_yawn = no_yawn + yawn

In [15]:
X = []
y = []

for features, label in combined_yawn:
    X.append(features)
    y.append(label)

In [22]:
X = np.array(X).reshape(-1, 145, 145, 3)
y = np.array(y)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [24]:
from keras.preprocessing.image import ImageDataGenerator
train_idg = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
test_idg = ImageDataGenerator(rescale=1/255)

In [28]:
train_generated = train_idg.flow(np.array(X_train), y_train, shuffle=False)
test_generated = test_idg.flow(np.array(X_test), y_test, shuffle=False)

In [38]:
len(test_generated)

5

In [39]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [40]:
model = Sequential()

model.add(Conv2D(256, (3, 3), activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="softmax"))

model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 143, 143, 256)     7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 71, 71, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 69, 69, 128)       295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 34, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        1

In [ ]:
history = model.fit(train_generator, epochs=300, validation_data=test_generator, shuffle=True, validation_steps=len(test_generator))